# 03. 시각화 (Visualization)

성격 척도 간의 상관관계와 분포를 시각화합니다.

## 학습 목표
- 상관행렬 히트맵 해석
- 성격 척도 분포 특성 이해
- Pairwise N 개념 이해

In [ ]:
# 필요한 라이브러리 설치 (처음 한 번만 실행)
%pip install pandas numpy matplotlib seaborn -q

In [ ]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 상위 폴더로 이동
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
print(f'작업 폴더: {os.getcwd()}')

# 그래프 스타일 설정
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

## 1. 데이터 로드

In [ ]:
# 전처리된 점수 데이터 로드
scores = pd.read_csv('data/processed/sapa_scores.csv')
print(f'데이터 크기: {scores.shape}')
print(f'컬럼: {list(scores.columns)}')

In [ ]:
# 분석할 7개 척도
scale_cols = ['NEO_O', 'NEO_C', 'NEO_E', 'NEO_A', 'NEO_N', 'Ideology', 'Honesty_Humility']
scale_names = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism', 'Ideology', 'Honesty-Humility']

# 척도 데이터만 추출
df_scales = scores[scale_cols].copy()
df_scales.columns = scale_names

print('각 척도별 유효 N:')
for col in df_scales.columns:
    print(f'  {col}: {df_scales[col].notna().sum():,}')

## 2. 상관행렬 (Correlation Matrix)

7개 척도 간의 상관관계를 계산하고 히트맵으로 시각화합니다.

**Pairwise N**: 결측치가 다른 척도마다 다르므로, 각 상관계수는 두 척도 모두 유효한 값이 있는 사람들만 사용합니다.

In [ ]:
# 상관행렬 계산
corr_matrix = df_scales.corr()

print('=== 상관행렬 ===')
print(corr_matrix.round(2))

In [ ]:
# Pairwise N 계산 (각 쌍에서 유효한 관측치 수)
def pairwise_n(df):
    cols = df.columns
    n_matrix = pd.DataFrame(index=cols, columns=cols, dtype=int)
    for i in cols:
        for j in cols:
            n_matrix.loc[i, j] = (df[i].notna() & df[j].notna()).sum()
    return n_matrix

n_matrix = pairwise_n(df_scales)
print('=== Pairwise N ===')
print(n_matrix)

In [ ]:
# 상관행렬 히트맵
fig, ax = plt.subplots(figsize=(10, 8))

mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
sns.heatmap(corr_matrix, 
            annot=True, 
            fmt='.2f', 
            cmap='RdBu_r', 
            center=0,
            vmin=-1, 
            vmax=1,
            square=True,
            linewidths=0.5,
            ax=ax)

ax.set_title('Correlation Matrix of Personality Scales', fontsize=14, fontweight='bold')
plt.tight_layout()

# 저장
os.makedirs('reports/figures', exist_ok=True)
plt.savefig('reports/figures/correlation_matrix.png', dpi=150, bbox_inches='tight')
print('저장: reports/figures/correlation_matrix.png')
plt.show()

In [ ]:
# 주요 상관관계 추출 (|r| > 0.2)
print('=== 주요 상관관계 (|r| > 0.2) ===')
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        r = corr_matrix.iloc[i, j]
        if abs(r) > 0.2:
            col_i = corr_matrix.columns[i]
            col_j = corr_matrix.columns[j]
            n = n_matrix.loc[col_i, col_j]
            print(f'{col_i} - {col_j}: r = {r:.2f} (N = {n:,})')

## 3. Big Five 분포

In [ ]:
# Big Five 히스토그램
big_five = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism']

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.flatten()

for i, col in enumerate(big_five):
    ax = axes[i]
    data = df_scales[col].dropna()
    
    ax.hist(data, bins=30, edgecolor='white', alpha=0.7, color='steelblue')
    ax.axvline(data.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean={data.mean():.2f}')
    ax.set_title(col, fontsize=12, fontweight='bold')
    ax.set_xlabel('Score')
    ax.set_ylabel('Frequency')
    ax.legend(loc='upper right', fontsize=9)
    ax.set_xlim(1, 6)

# 마지막 빈 subplot 제거
axes[5].axis('off')

plt.suptitle('Big Five Personality Distributions', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()

plt.savefig('reports/figures/big_five_distributions.png', dpi=150, bbox_inches='tight')
print('저장: reports/figures/big_five_distributions.png')
plt.show()

## 4. Ideology & Honesty-Humility 분포

In [ ]:
# Ideology & H-H 히스토그램 (z-score 기반)
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Ideology
ax = axes[0]
data = df_scales['Ideology'].dropna()
ax.hist(data, bins=30, edgecolor='white', alpha=0.7, color='coral')
ax.axvline(data.mean(), color='darkred', linestyle='--', linewidth=2, label=f'Mean={data.mean():.2f}')
ax.set_title('Ideology (Conservative)', fontsize=12, fontweight='bold')
ax.set_xlabel('z-score')
ax.set_ylabel('Frequency')
ax.legend(loc='upper right')

# Honesty-Humility
ax = axes[1]
data = df_scales['Honesty-Humility'].dropna()
ax.hist(data, bins=30, edgecolor='white', alpha=0.7, color='mediumseagreen')
ax.axvline(data.mean(), color='darkgreen', linestyle='--', linewidth=2, label=f'Mean={data.mean():.2f}')
ax.set_title('Honesty-Humility', fontsize=12, fontweight='bold')
ax.set_xlabel('z-score')
ax.set_ylabel('Frequency')
ax.legend(loc='upper right')

plt.suptitle('Composite Scale Distributions', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()

plt.savefig('reports/figures/ideology_hh_distributions.png', dpi=150, bbox_inches='tight')
print('저장: reports/figures/ideology_hh_distributions.png')
plt.show()

## 5. 최종 요약

In [ ]:
print('=== 시각화 완료 요약 ===')
print('\n[생성된 그림]')
print('  1. reports/figures/correlation_matrix.png')
print('  2. reports/figures/big_five_distributions.png')
print('  3. reports/figures/ideology_hh_distributions.png')

print('\n[주요 상관관계]')
# 가장 강한 양의 상관
corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        r = corr_matrix.iloc[i, j]
        corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], r))

corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)

print('  가장 강한 상관 Top 5:')
for col_i, col_j, r in corr_pairs[:5]:
    direction = '양' if r > 0 else '음'
    print(f'    {col_i} - {col_j}: r = {r:.2f} ({direction}의 상관)')